In [4]:
import coin as c
import numpy as np
import payment_managment as pm
from decimal import *
from ticket import *
import exceptions
import gui

In [5]:
class Ticket_Machine(pm.Payment_Management):
    """Klasa obsługująca logikę działania automatu biletowego"""
    def __init__(self):
        super().__init__()
#         gui.GUI(self)
        
        
    def create_add_ticket(self, value: float):
        """Zwraca funckję naliczającą opłatę za konkretny bilet"""
        ticket = Ticket(value)
        def add_ticket() -> None:
            """Nalicza opłatę za bilet"""
            self.credit += ticket.get_price
        return add_ticket
    
    def resign(self) -> None:
        """Funckja resetująca"""
        self.lista=np.array([])
        self.credit = 0
        self.payment = 0
        self.gui.update_credit_status()
        self.gui.display_paymnet_layout = False
        
    def addCoinsToPM(self, entry):
        type(entry)
        added = np.array([])
        for e in range(len(entry)): #numer wejścia
            for i in range((int)(entry[e])): #po danych
                self.add_coin(self.expectedValue[e])
                added = np.append(added, c.Coin(self.expectedValue[e]))
        return added
                
    def checkData(self, entry):
        for e in entry:
            if not e.isnumeric():
                return False
        return True
        
    def manageChange(self, entry = False):
        added = None
        try:
            if entry == False:
                entry = self.gui.readDataFromEntries()
            type(entry)
            if self.checkData(entry):
                added = self.addCoinsToPM(entry)

                if self.get_amount_to_pay < 0:
                    self.resign()
                    raise exceptions.NotEnoughMoney(self.gui.window)
                    return

                if self.get_amount_to_pay == 0:
                    raise exceptions.NoChangeNeeded(self.gui.window)
                    return

                result = self.change(self.get_amount_to_pay)

                if type(result) == type(False):
                    
                    for coin in added:
                        self.return_coin(coin)
                    raise exceptions.CannotChange(self.gui.window)
                
                else:
                    print(result)
                    self.resign()
                    return result
            else:
                self.resign()
                raise exceptions.InvalidArgument(self.gui.window)
        except exceptions.NoChangeNeeded as exc:
            print(exc)
            return added

        
        
        


In [6]:
app = Ticket_Machine()
app.credit = 6
print(app.manageChange(["0","0","0","1","1","1","1","0","0","1","1","0"]))

[2.00 0.50 0.05 0.02]


NameError: name 'TclError' is not defined

In [ ]:
import unittest

class TestPM(unittest.TestCase):
    def setUp(self):
        self.app = Ticket_Machine()
        
    def test_shouldRaiseNoChangeNeeded_whenPaidActualAmount(self) -> None:
        self.app.credit = 5
        self.assertRaises(exceptions.NoChangeNeeded, self.app.manageChange, ["0","0","0","1","0","0","0","0","0","0","0","0"])
        
    def test_shouldGiveCorrectChange_WhenPossible(self) -> None:
        self.app.credit = 5
        result = self.app.manageChange(["0","0","0","1","1","0","0","0","0","0","0","0"])
        expected = np.array([c.Coin(Decimal(2).quantize(Decimal('0.00')))])

        np.testing.assert_array_equal(result, expected)

        
    def test_shouldReturnAddedCoin_whenCannotChange(self) -> None:
        self.app.credit = 6
        print(self.app.manageChange(["0","0","0","1","1","1","1","0","0","1","1","0"]))
        
if __name__ == '__main__':
    unittest.main()